### Set up the Spark environment.

In [1]:
import subprocess
result = subprocess.run(["gcloud", "config", "get-value", "project"], stdout=subprocess.PIPE)
PROJECT_ID = result.stdout.decode("utf-8").strip()

# Print the output
print(PROJECT_ID)

qwiklabs-gcp-00-21f5cc02ae3a


In [2]:
SOURCE_PATH = f'gs://{PROJECT_ID}-data/datasets'
DEST_PATH = f'gs://{PROJECT_ID}-output'
print(SOURCE_PATH, DEST_PATH)

gs://qwiklabs-gcp-00-21f5cc02ae3a-data/datasets gs://qwiklabs-gcp-00-21f5cc02ae3a-output


In [4]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

prodSchema = StructType([
    StructField('productid', IntegerType()), 
    StructField('productname', StringType()),
    StructField('supplierid', IntegerType()), 
    StructField('categoryid', IntegerType()), 
    StructField('quantityperunit', StringType()), 
    StructField('unitprice', FloatType()), 
    StructField('unitsinstock', IntegerType()), 
    StructField('unitsonorder', IntegerType()), 
    StructField('reorderlevel', IntegerType()), 
    StructField('discontinued', IntegerType())
])
prod = spark.read.json(f'{SOURCE_PATH}/northwind/JSON/products', schema=prodSchema)
prod.show()


+---------+--------------------+----------+----------+--------------------+---------+------------+------------+------------+------------+
|productid|         productname|supplierid|categoryid|     quantityperunit|unitprice|unitsinstock|unitsonorder|reorderlevel|discontinued|
+---------+--------------------+----------+----------+--------------------+---------+------------+------------+------------+------------+
|        1|                Chai|         8|         1|  10 boxes x 30 bags|     18.0|          39|           0|          10|           1|
|        2|               Chang|         1|         1|  24 - 12 oz bottles|     19.0|          17|          40|          25|           1|
|        3|       Aniseed Syrup|         1|         2| 12 - 550 ml bottles|     10.0|          13|          70|          25|           0|
|        4|Chef Anton's Caju...|         2|         2|      48 - 6 oz jars|     22.0|          53|           0|           0|           0|
|        5|Chef Anton's Gumb...|  

## Spark DataFrames can be saved to a Hive table using either the saveAsTable method

In [8]:
prod.write \
    .mode("overwrite") \
    .saveAsTable("products")


ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
25/01/16 20:18:45 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


## You can query an existing Hive table and bring it into a Spark DataFrame.

In [9]:
prod2 = spark.sql('select * from products')
print(prod2)
prod2.show()


DataFrame[productid: int, productname: string, supplierid: int, categoryid: int, quantityperunit: string, unitprice: float, unitsinstock: int, unitsonorder: int, reorderlevel: int, discontinued: int]
+---------+--------------------+----------+----------+--------------------+---------+------------+------------+------------+------------+
|productid|         productname|supplierid|categoryid|     quantityperunit|unitprice|unitsinstock|unitsonorder|reorderlevel|discontinued|
+---------+--------------------+----------+----------+--------------------+---------+------------+------------+------------+------------+
|        1|                Chai|         8|         1|  10 boxes x 30 bags|     18.0|          39|           0|          10|           1|
|        2|               Chang|         1|         1|  24 - 12 oz bottles|     19.0|          17|          40|          25|           1|
|        3|       Aniseed Syrup|         1|         2| 12 - 550 ml bottles|     10.0|          13|          70

## Read in a file to a Spark DataFrame.

In [11]:
categories = spark.read.csv(f'{SOURCE_PATH}/northwind/CSVHeaders/categories', header=True)
categories.show()


+----------+--------------+--------------------+
|CategoryID|  CategoryName|         Description|
+----------+--------------+--------------------+
|         1|     Beverages|Soft drinks coffe...|
|         2|    Condiments|Sweet and savory ...|
|         3|   Confections|Desserts candies ...|
|         4|Dairy Products|             Cheeses|
|         5|Grains/Cereals|Breads crackers p...|
|         6|  Meat/Poultry|      Prepared meats|
|         7|       Produce|Dried fruit and b...|
|         8|       Seafood|    Seaweed and fish|
+----------+--------------+--------------------+



## Use createOrReplaceTempView to create a virtual table in the Hive catalog and then it can be queried using SQL as if it were a hive table.

In [12]:
categories.createOrReplaceTempView('categories')
t1 =spark.sql('select * from categories where categoryid = 1')
t1.show()
print(t1.count())


+----------+------------+--------------------+
|CategoryID|CategoryName|         Description|
+----------+------------+--------------------+
|         1|   Beverages|Soft drinks coffe...|
+----------+------------+--------------------+

1


### The create table as syntax also works

In [ ]:
spark.sql('create table categories2 as select * from categories')


## Queries use standard HQL to mix Hive tables and virtual tables. Both are read into a Spark DataFrame and the processing happens at the Spark level, not at the Hive level. HQL is just used to parse the logic into the corresponding Spark methods.

In [18]:
sql = """
SELECT c.categoryid, c.categoryname, p.productid, p.productname
from categories as c
join products as p on c.categoryid = p.categoryid
order by c.categoryid, p.productid
"""
df = spark.sql(sql)
# df.show(10)

df2 = categories.join(prod, categories.CategoryID == prod.productid) \
                .select(categories.CategoryID, 'categoryname', 'productid', 'productname') \
                .orderBy(categories.CategoryID, 'productid')

df2.show(10)



+----------+--------------+---------+--------------------+
|CategoryID|  categoryname|productid|         productname|
+----------+--------------+---------+--------------------+
|         1|     Beverages|        1|                Chai|
|         2|    Condiments|        2|               Chang|
|         3|   Confections|        3|       Aniseed Syrup|
|         4|Dairy Products|        4|Chef Anton's Caju...|
|         5|Grains/Cereals|        5|Chef Anton's Gumb...|
|         6|  Meat/Poultry|        6|Grandma's Boysenb...|
|         7|       Produce|        7|Uncle Bob's Organ...|
|         8|       Seafood|        8|Northwoods Cranbe...|
+----------+--------------+---------+--------------------+



## Creating the regions2 DataFrame does not execute anything yet, but by making the DataFrame into a Temp View then running a Spark SQL query, it tells Spark to read the SQL data into a DataFrame and then use the cluster to do the processing, not the SQL source.

In [ ]:
regions2.createOrReplaceTempView('regions2')
spark.sql('select * from regions2 where regionid < 3').show()

In [ ]:
spark.read.table('regions2').where('regionid < 3').show()

## Alternate ways to code a query using SQL and methods.

In [ ]:
print(spark.sql('select count(*) from regions').collect())
spark.sql('select * from regions').count()

## Using SQL you can use familiar syntax instead of withColumn or withColumnRenamed methods.
Note the expr function needs to be imported when you want to use a stringified SQL function using dot syntax.

In [ ]:
from pyspark.sql.functions import expr

t1 = spark.sql('select TerritoryID as TerrID, UPPER(TerritoryName) as TerritoryName, RegionID from territories')
t1.show(5)

from pyspark.sql.functions import expr
territories.withColumn('TerritoryName', expr('UPPER(TerritoryName)')).withColumnRenamed('TerritoryID', 'TerrID').show(5)


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

# This won't work though if you want to use python functions, you need to go another step
# territories.withColumn('TerritoryName', territories.TerritoryName.upper()).show()

# You need to make the python function callable by spark by wrapping it in the udf function
# which tells spark what datatype it returns
territories.withColumn('TerritoryName', udf(str.upper, StringType())(territories.TerritoryName)).show()



## If you want to use a function that is not a standard Python or SQL function, you can always create one in Python and make it callable from Spark.

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

def inventoryvalue(quantity, price):
    return quantity * price

# Turn the Python function into a Spark callable function
invvalue = udf(inventoryvalue, FloatType())
p = products
p2 = p.withColumn('value', invvalue(p.unitsinstock, p.unitprice))
display(p2)




## Python decorators are an even better option.

In [ ]:
@udf(FloatType())
def inventoryvalue(quantity, price):
    return quantity * price

p2 = p.withColumn('value', inventoryvalue(p.unitsinstock, p.unitprice))
display(p2)


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

def inventoryvalue(quantity, price):
    return quantity * price

# Or dynamically wrap it, but it's harder to read
p2 = p.withColumn('value', udf(inventoryvalue, FloatType())(p.unitsinstock, p.unitprice))
display(p2)



## To make it easier though, you could make the Python function into a udf that SQL can understand similar to how you can make a DataFrame seem like a virtual table with createOrReplaceTempView.

In [ ]:
def reverseString(x):
    return x[::-1]

spark.udf.register('reverse', reverseString, StringType())

spark.sql('select *, reverse(TerritoryName) as Reversed from Territories').orderBy('Reversed').show()

## HQL has collect_set and collect_list functions to aggregate items into a list instead of summing them up. 

In [ ]:
from pyspark.sql.functions import collect_list
territories.groupBy(territories.RegionID).agg(collect_list(territories.TerritoryName)).show()

tr1 = spark.sql("SELECT RegionID, collect_list(TerritoryName) AS TerritoryList FROM Territories GROUP BY RegionID")
tr1.show()
tr1.printSchema()
print(tr1.take(1))


## Instead of a simple datatype, you could also collect complex structured objects using the HQL NAMED_STRUCT.

In [ ]:
sql = """
SELECT r.RegionID, r.RegionName
, COLLECT_SET(NAMED_STRUCT("TerritoryID", TerritoryID, "TerritoryName", TerritoryName)) AS TerritoryList
FROM Regions AS r
JOIN Territories AS t ON r.RegionID = t.RegionID
GROUP BY r.RegionID, r.RegionName
ORDER BY r.RegionID
"""

tr2 = spark.sql(sql)
tr2.printSchema()
print(tr2)
tr2.show()
print(tr2.take(2))
tr2.write.json('TerritoryRegion.json')
spark.sql('create table TerritoryRegion as ' + sql)

## If you have data that is already collected into a complex datatype and want to flatten it, you could use HQL EXPLODE function.

## You could use the Spark explode method.

In [ ]:
from pyspark.sql.functions import explode
tr1.select('RegionID', explode('TerritoryList')).show()


## Or if the DataFrame is turned into a Temp View, you could use the HQL query to do it.

In [ ]:
tr1.createOrReplaceTempView('RegionTerritories')
sql = """
SELECT RegionID, TerritoryName
FROM RegionTerritories
LATERAL VIEW EXPLODE(TerritoryList) EXPLODED_TABLE AS TerritoryName
ORDER BY RegionID, TerritoryName
"""
spark.sql(sql).show()

## Or you could select specific elements from a collection.

In [ ]:
tr2.createOrReplaceTempView('RegionTerritories')
spark.sql("select RegionId, RegionName, TerritoryList[0] as First, TerritoryList[size(TerritoryList) - 1] as Last, size(TerritoryList) as TerritoryCount from RegionTerritories").show()


## If the array is of structs, note the syntax of fetching the elements from the struct uses the . like an object property.

In [ ]:
sql = """
SELECT RegionID, RegionName, Territory.TerritoryID AS TerritoryID
, Territory.TerritoryName AS TerritoryName
FROM RegionTerritories
LATERAL VIEW EXPLODE(TerritoryList) EXPLODED_TABLE AS Territory
"""
spark.sql(sql).show()


## HOMEWORK: ## 
**First Challenge**

Create a Python function to determine if a number is odd or even and use that to select only the even numbered shippers from the TSV folder of northwind. Note the TSV file does not have headers so you will need to do something to make the DataFrame have a meaningful structure. I would suggest using Spark SQL as much as possible to rename and cast the columns which are ShipperID, CompanyName, and Phone.

**Second Challenge**

Take the Order_LineItems.json folder, read it into a DataFrame, and flatten it and then calculate the average price paid for a product.

<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Take a look at the MakeOrders_LineItems.py file provided to see how the Order_LineItems.json was generated in the first place
<br>
Use modulus with remainder of zero to determine if something is even
<br>
Use udf to make a version of the function that is callable using dot syntax and udf.register to make a version callable from within a SQL string
<br>
Use LATERAL VIEW EXPLODE() EXPLODED_TABLE to flatten out the nested format file
<br>
Once flattened do a traditional aggregate to calculate the average
<br>
<br>
</p>
</details>



